In [3]:
import pandas as pd
import requests
import json
import glob
import os
import urllib3
import httpx
import time
import shutil

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [4]:
# Start the session
session = requests.Session()
access_token = "0d1aa8045931ca9fa438fbc41e09bad91fefd1a1"

headers = {
    "Authorization": "Bearer " + access_token
}

## ENTER YEAR AND ROUND NUMBER

In [5]:
# specify year
year_number = 2024

# specify round
round_number = 24

## PLAYER LIST

In [6]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [9]:
s = session.get("https://www.supercoach.com.au/2024/api/afl/draft/v1/players-cf?embed=notes%2Codds%2Cplayer_stats%2Cpositions&round={}".format(round_number), headers={**headers, 'User-Agent': 'Mozilla/5.0'}, verify=False)
if s.status_code == 200:
    player_json = json.loads(s.text)
else:
    raise Exception(f'Error fetching data: {s.status_code} - {s.text}')
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'player_id': [player['player_stats'][0]['player_id']],
            'round': [player['player_stats'][0]['round']],
            'points': [player['player_stats'][0]['points']],
            'price': [player['player_stats'][0]['price']],
            'total_points': [player['player_stats'][0]['total_points']],
            'position': [player['player_stats'][0]['position']],
            'games': [player['player_stats'][0]['games']],
            'total_games': [player['player_stats'][0]['total_games']],
            'price_change': [player['player_stats'][0]['price_change']],
            'total_price_change': [player['player_stats'][0]['total_price_change']],
            'last_position': [player['player_stats'][0]['last_position']],
            'round_position': [player['player_stats'][0]['round_position']],
            'minutes_played': [player['player_stats'][0]['minutes_played']],
            'total_minutes_played': [player['player_stats'][0]['total_minutes_played']],
            'updated_at': [player['player_stats'][0]['updated_at']],
            'kicks': [player['player_stats'][0]['kicks']],
            'total_kicks': [player['player_stats'][0]['total_kicks']],
            'handballs': [player['player_stats'][0]['handballs']],
            'total_handballs': [player['player_stats'][0]['total_handballs']],
            'marks': [player['player_stats'][0]['marks']],
            'total_marks': [player['player_stats'][0]['total_marks']],
            'tackles': [player['player_stats'][0]['tackles']],
            'total_tackles': [player['player_stats'][0]['total_tackles']],
            'freekicks_for': [player['player_stats'][0]['freekicks_for']],
            'total_freekicks_for': [player['player_stats'][0]['total_freekicks_for']],
            'freekicks_against': [player['player_stats'][0]['freekicks_against']],
            'total_freekicks_against': [player['player_stats'][0]['total_freekicks_against']],
            'hitouts': [player['player_stats'][0]['hitouts']],
            'total_hitouts': [player['player_stats'][0]['total_hitouts']],
            'goals': [player['player_stats'][0]['goals']],
            'total_goals': [player['player_stats'][0]['total_goals']],
            'behinds': [player['player_stats'][0]['behinds']],
            'total_behinds': [player['player_stats'][0]['total_behinds']],
            'avg5': [player['player_stats'][0]['avg5']],
            'own_raw': [player['player_stats'][0]['own_raw']],
            'avg': [player['player_stats'][0]['avg']],
            'avg3': [player['player_stats'][0]['avg3']],
            'opp': [player['player_stats'][0]['opp']['abbrev']],
            'oppavg': [player['player_stats'][0]['oppavg']],
            'opph': [player['player_stats'][0]['opph']],
            'opp1': [player['player_stats'][0]['opp1']],
            'opp1h': [player['player_stats'][0]['opp1h']],
            'opp2': [player['player_stats'][0]['opp2']],
            'opp2h': [player['player_stats'][0]['opp2h']],
            'opp3': [player['player_stats'][0]['opp3']],
            'opp3h': [player['player_stats'][0]['opp3h']],
            'ven': [player['player_stats'][0]['ven']['name']],
            'venavg': [player['player_stats'][0]['venavg']],
            'ven1': [player['player_stats'][0]['ven1']],
            'ven2': [player['player_stats'][0]['ven2']],
            'ven3': [player['player_stats'][0]['ven3']],
            'livepts': [player['player_stats'][0]['livepts']],
            'livegames': [player['player_stats'][0]['livegames']],
            'owned': [player['player_stats'][0]['owned']],
            'points_per_min': [player['player_stats'][0]['points_per_min']],
            'total_points_per_min': [player['player_stats'][0]['total_points_per_min']],
            'mvp_value': [player['player_stats'][0]['mvp_value']],
            'position_change': [player['player_stats'][0]['position_change']],
            # 'position_ranks': [player['player_stats'][0]['position_ranks']]
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list.to_csv('outputs/player_list.csv')
    
df_player_list

,player_id,feed_id,first_name,last_name,team,position,round,points,price,total_points,...,ven1,ven2,ven3,livepts,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change
0,695,297373,Marcus,Bontempelli,WBD,1,24,116,665800,2907,...,None,None,None,0,0,99.826500,1.104762,1.209738,5267.780143,0
0,706,1004592,Tim,English,WBD,31,24,103,521900,2376,...,None,None,None,0,0,95.861200,0.971698,1.016688,4832.407407,1
0,458,996701,Clayton,Oliver,MEL,114,24,0,404900,1760,...,None,None,None,0,0,86.071900,None,0.88044,4831.194554,-15
0,459,298210,Christian,Petracca,MEL,238,24,0,519000,1342,...,None,None,None,0,0,62.057000,None,1.084006,5027.559551,-12
0,20,293222,Rory,Laird,ADE,30,24,96,516600,2395,...,None,None,None,0,0,98.190800,0.864865,1.080776,4961.106309,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,161,1005107,Nathan,*** Murphy,COL,658,24,0,340600,0,...,None,None,None,0,0,1.239160,None,None,None,-2
0,684,290188,Sam,*** Reid,SYD,658,24,0,286800,0,...,None,None,None,0,0,0.743494,None,None,None,-2
0,134,1027021,Josh,*** Carmichael,COL,658,24,0,123900,0,...,None,None,None,0,0,0.099133,None,None,None,-2
0,723,1024064,Aiden,*** O'Driscoll,WBD,658,24,0,117300,0,...,None,None,None,0,0,0.495663,None,None,None,-2


# Get player stats for every game

In [11]:
# Define your headers with authentication token
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Authorization': 'Bearer ' + access_token  # Replace with your actual access token
}

# Create the directory if it doesn't exist
output_dir = 'inputs/player_stats_current'
os.makedirs(output_dir, exist_ok=True)

# Get the total number of players
total_players = len(df_player_list)
print(f"Total number of players: {total_players}")

# Iterate through each player in df_player_list using enumerate to get the correct index
for index, row in enumerate(df_player_list.itertuples(), start=1):
    player_id = row.player_id
    url = f"https://www.supercoach.com.au/2024/api/afl/draft/v1/players/{player_id}?embed=notes,odds,player_stats,player_match_stats,positions,trades"
    
    # Define the output file path
    output_file = os.path.join(output_dir, f'player_stats_current_season_{player_id}.json')
    
    # Check if the output file already exists
    if not os.path.exists(output_file):
        # Make the request to the URL
        response = httpx.get(url, headers=headers, verify=False)
        
        if response.status_code == 200:
            player_stats = response.json()
            # Save the response to a file
            with open(output_file, 'w') as f:
                json.dump(player_stats, f)
            
            # Calculate and print the completion percentage within the same print statement
            completion_percentage = (index / total_players) * 100
            print(f"Player ID: {player_id}, Stats saved to {output_file}, Completion: {completion_percentage:.2f}%")
        else:
            print(f"Error fetching data for Player ID: {player_id}, Status Code: {response.status_code}")
    else:
        print(f"Data for Player ID: {player_id} already exists in {output_file}")

Total number of players: 808
Data for Player ID: 695 already exists in inputs/player_stats_current\player_stats_current_season_695.json
Data for Player ID: 706 already exists in inputs/player_stats_current\player_stats_current_season_706.json
Data for Player ID: 458 already exists in inputs/player_stats_current\player_stats_current_season_458.json
Data for Player ID: 459 already exists in inputs/player_stats_current\player_stats_current_season_459.json
Data for Player ID: 20 already exists in inputs/player_stats_current\player_stats_current_season_20.json
Data for Player ID: 197 already exists in inputs/player_stats_current\player_stats_current_season_197.json
Data for Player ID: 138 already exists in inputs/player_stats_current\player_stats_current_season_138.json
Data for Player ID: 717 already exists in inputs/player_stats_current\player_stats_current_season_717.json
Data for Player ID: 9 already exists in inputs/player_stats_current\player_stats_current_season_9.json
Data for Playe

## extract relevant data from saved files above

In [40]:
# Load the JSON data from the file
with open('inputs/player_stats_current/player_stats_current_season_695.json', 'r') as file:
    data = json.load(file)

# Extract the relevant fields into a list of dictionaries
player_match_stats = data['player_match_stats']
extracted_data = []
for match in player_match_stats:
    round_number = match['round']
    extracted_data.append({
        'feed_id': data['feed_id'],
        'player_id': match['player_id'],
        'first_name': data['first_name'],
        'last_name': data['last_name'],
        'team_abbrev': data['team']['abbrev'],
        'pos_1': data['positions'][0]['position'],
        'pos_2': data['positions'][1]['position'] if len(data['positions']) > 1 else None,
        'round': match['round'],
        'points': match['points'],
        'played': data['player_stats'][round_number]['total_games'],
        'avg': data['player_stats'][round_number]['avg'],
        'avg3': data['player_stats'][round_number]['avg3'],
        'avg5': data['player_stats'][round_number]['avg5'],
        'price': data['player_stats'][round_number]['price']
    })

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Display the DataFrame
display(df)

,feed_id,player_id,first_name,last_name,team_abbrev,pos_1,pos_2,round,points,played,avg,avg3,avg5,price
0,297373,695,Marcus,Bontempelli,WBD,MID,None,1,126,1,126.000,126.000,126.00,724600
1,297373,695,Marcus,Bontempelli,WBD,MID,None,2,136,2,131.000,131.000,131.00,724600
2,297373,695,Marcus,Bontempelli,WBD,MID,None,3,110,3,124.000,124.000,124.00,704900
3,297373,695,Marcus,Bontempelli,WBD,MID,None,4,141,4,128.250,129.000,128.25,697700
4,297373,695,Marcus,Bontempelli,WBD,MID,None,5,73,5,117.200,108.000,117.20,664500
5,297373,695,Marcus,Bontempelli,WBD,MID,None,6,125,6,118.500,113.000,117.00,646000
6,297373,695,Marcus,Bontempelli,WBD,MID,None,7,102,7,116.143,100.000,110.20,614000
7,297373,695,Marcus,Bontempelli,WBD,MID,None,8,142,8,119.375,123.000,116.60,623100
8,297373,695,Marcus,Bontempelli,WBD,MID,None,9,135,9,121.111,126.333,115.40,634300
9,297373,695,Marcus,Bontempelli,WBD,MID,None,10,93,10,118.300,123.333,119.40,635800


# Get complete player stats pack for each player for past seasons

In [12]:
# Define your headers with authentication token
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Authorization': 'Bearer ' + access_token  # Replace with your actual access token
}

# Create the directory if it doesn't exist
output_dir = 'inputs/player_stats_pack'
os.makedirs(output_dir, exist_ok=True)

# Get the total number of players
total_players = len(df_player_list)
print(f"Total number of players: {total_players}")

# Iterate through each player in df_player_list using enumerate to get the correct index
for index, row in enumerate(df_player_list.itertuples(), start=1):
    player_id = row.player_id
    url = f"https://www.supercoach.com.au/2024/api/afl/draft/v1/completeStatspack?player_id={player_id}"
    
    # Define the output file path
    output_file = os.path.join(output_dir, f'completestatspack_{player_id}.json')
    
    # Check if the output file already exists
    if not os.path.exists(output_file):
        # Make the request to the URL
        response = httpx.get(url, headers=headers, verify=False)
        
        if response.status_code == 200:
            player_stats = response.json()
            # Save the response to a file
            with open(output_file, 'w') as f:
                json.dump(player_stats, f)
            
            # Calculate and print the completion percentage within the same print statement
            completion_percentage = (index / total_players) * 100
            print(f"Player ID: {player_id}, Stats saved to {output_file}, Completion: {completion_percentage:.2f}%")
        else:
            print(f"Error fetching data for Player ID: {player_id}, Status Code: {response.status_code}")
    else:
        print(f"Data for Player ID: {player_id} already exists in {output_file}")

Total number of players: 808
Data for Player ID: 695 already exists in inputs/player_stats_pack\completestatspack_695.json
Data for Player ID: 706 already exists in inputs/player_stats_pack\completestatspack_706.json
Data for Player ID: 458 already exists in inputs/player_stats_pack\completestatspack_458.json
Data for Player ID: 459 already exists in inputs/player_stats_pack\completestatspack_459.json
Data for Player ID: 20 already exists in inputs/player_stats_pack\completestatspack_20.json
Data for Player ID: 197 already exists in inputs/player_stats_pack\completestatspack_197.json
Data for Player ID: 138 already exists in inputs/player_stats_pack\completestatspack_138.json
Data for Player ID: 717 already exists in inputs/player_stats_pack\completestatspack_717.json
Data for Player ID: 9 already exists in inputs/player_stats_pack\completestatspack_9.json
Data for Player ID: 54 already exists in inputs/player_stats_pack\completestatspack_54.json
Data for Player ID: 629 already exists 

## LADDER AND FIXTURE DATA
- also saves each json file

In [20]:
match_json_list = []

for rd in range(1, round_number + 1):
    s = httpx.get(
        "https://www.supercoach.com.au/2024/api/afl/draft/v1/leagues/1283/ladderAndFixtures?round={}&scores=true".format(rd), headers={**headers, 'User-Agent': 'Mozilla/5.0'}, verify=False)
    if s.status_code == 200:
        try:
            match_json = json.loads(s.text)
            match_json_list.append(match_json)
            with open("outputs/json files/match_json_rd{}.json".format(rd), 'w') as outfile:
                json.dump(match_json, outfile, indent=4, sort_keys=True)
            progress = (rd / round_number) * 100
            print(f"Successfully fetched and saved data for round {rd} ({progress:.2f}% complete)")
        except json.JSONDecodeError:
            print(f"Error decoding JSON for round {rd}")
    else:
        print(f"Failed to fetch data for round {rd}, status code: {s.status_code}")

Successfully fetched and saved data for round 1 (4.17% complete)
Successfully fetched and saved data for round 2 (8.33% complete)
Successfully fetched and saved data for round 3 (12.50% complete)
Successfully fetched and saved data for round 4 (16.67% complete)
Successfully fetched and saved data for round 5 (20.83% complete)
Successfully fetched and saved data for round 6 (25.00% complete)
Successfully fetched and saved data for round 7 (29.17% complete)
Successfully fetched and saved data for round 8 (33.33% complete)
Successfully fetched and saved data for round 9 (37.50% complete)
Successfully fetched and saved data for round 10 (41.67% complete)
Successfully fetched and saved data for round 11 (45.83% complete)
Successfully fetched and saved data for round 12 (50.00% complete)
Successfully fetched and saved data for round 13 (54.17% complete)
Successfully fetched and saved data for round 14 (58.33% complete)
Successfully fetched and saved data for round 15 (62.50% complete)
Succes

## COACH IDs AND NAMES

In [21]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,71052,716,James,The Gulden Girls
0,300791,713,Paul,Toby or Not To Be
0,45168,719,Luke,He is HIMmelberg
0,8258,718,Simon,Wilkie Wonka
0,180650,720,Jordan,StupidSexyFlanders
0,21562,714,Mark,Need for Sheed
0,10857,715,Anthony,Colonel Sanders
0,86078,717,Lester,Baby Got Trac


In [22]:
df_coaches.to_csv('inputs/coach_list.csv')

## PLAYER MATCH RESULTS
Function to get player scores for each match

In [23]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [24]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
display(df_player_match_results)
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round_x'].astype('str')
df_player_match_results

,round_x,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points_x,on_field,feed_id,...,ven1,ven2,ven3,livepts,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change
0,1,71052,716,James,The Gulden Girls,590,RUC,137,True,298174,...,None,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1
1,2,71052,716,James,The Gulden Girls,590,RUC,96,True,298174,...,None,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1
2,3,71052,716,James,The Gulden Girls,590,RUC,73,True,298174,...,None,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1
3,4,71052,716,James,The Gulden Girls,590,RUC,133,True,298174,...,None,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1
4,5,71052,716,James,The Gulden Girls,590,RUC,87,True,298174,...,None,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,22,300791,713,Paul,Toby or Not To Be,713,FWD,0,False,1020586,...,None,None,None,0,0,0.495663,0.47619,0.47619,4772.5,22
4412,23,8258,718,Simon,Wilkie Wonka,171,RUC,85,True,1011954,...,None,None,None,0,0,5.452290,0.59434,0.832941,5145.480226,18
4413,24,8258,718,Simon,Wilkie Wonka,171,RUC,63,False,1011954,...,None,None,None,0,0,5.452290,0.59434,0.832941,5145.480226,18
4414,24,300791,713,Paul,Toby or Not To Be,216,MID,0,False,998129,...,None,None,None,0,0,4.560100,None,None,None,-2


,round_x,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points_x,on_field,feed_id,...,ven2,ven3,livepts,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change,round_number_alt
0,1,71052,716,James,The Gulden Girls,590,RUC,137,True,298174,...,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1,R1
1,2,71052,716,James,The Gulden Girls,590,RUC,96,True,298174,...,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1,R2
2,3,71052,716,James,The Gulden Girls,590,RUC,73,True,298174,...,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1,R3
3,4,71052,716,James,The Gulden Girls,590,RUC,133,True,298174,...,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1,R4
4,5,71052,716,James,The Gulden Girls,590,RUC,87,True,298174,...,None,None,0,0,91.251500,1.115789,1.120523,5049.982765,-1,R5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,22,300791,713,Paul,Toby or Not To Be,713,FWD,0,False,1020586,...,None,None,0,0,0.495663,0.47619,0.47619,4772.5,22,R22
4412,23,8258,718,Simon,Wilkie Wonka,171,RUC,85,True,1011954,...,None,None,0,0,5.452290,0.59434,0.832941,5145.480226,18,R23
4413,24,8258,718,Simon,Wilkie Wonka,171,RUC,63,False,1011954,...,None,None,0,0,5.452290,0.59434,0.832941,5145.480226,18,R24
4414,24,300791,713,Paul,Toby or Not To Be,216,MID,0,False,998129,...,None,None,0,0,4.560100,None,None,None,-2,R24


In [25]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [26]:
df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,71052,716,James,The Gulden Girls,1,0,0,1946,1559,4,1
0,1,300791,713,Paul,Toby or Not To Be,1,0,0,1884,1640,4,2
0,1,45168,719,Luke,He is HIMmelberg,1,0,0,1872,1682,4,3
0,1,8258,718,Simon,Wilkie Wonka,1,0,0,1819,1529,4,4
0,1,180650,720,Jordan,StupidSexyFlanders,0,0,1,1682,1872,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,24,45168,719,Luke,He is HIMmelberg,0,0,0,0,0,0,4
0,24,86078,717,Lester,Baby Got Trac,0,0,0,0,0,0,5
0,24,10857,715,Anthony,Colonel Sanders,0,0,0,0,0,0,6
0,24,71052,716,James,The Gulden Girls,0,0,0,0,0,0,7


In [27]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [28]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,713,Toby or Not To Be,Paul,1884,714,Need for Sheed,Mark,1640
0,1,Anthony vs James,715,Colonel Sanders,Anthony,1559,716,The Gulden Girls,James,1946
0,1,Lester vs Simon,717,Baby Got Trac,Lester,1529,718,Wilkie Wonka,Simon,1819
0,1,Luke vs Jordan,719,He is HIMmelberg,Luke,1872,720,StupidSexyFlanders,Jordan,1682
0,2,Paul vs Anthony,713,Toby or Not To Be,Paul,1851,715,Colonel Sanders,Anthony,1596
...,...,...,...,...,...,...,...,...,...,...
0,21,Lester vs Luke,717,Baby Got Trac,Lester,1771,719,He is HIMmelberg,Luke,1720
0,22,Jordan vs Paul,720,StupidSexyFlanders,Jordan,1732,713,Toby or Not To Be,Paul,1878
0,22,Luke vs Simon,719,He is HIMmelberg,Luke,1641,718,Wilkie Wonka,Simon,1702
0,23,Jordan vs Simon,720,StupidSexyFlanders,Jordan,1930,718,Wilkie Wonka,Simon,1769


In [29]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [30]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,Anthony vs James,716,The Gulden Girls,James,1946,715,Colonel Sanders,Anthony,1559,1.0,7.0
0,1,Paul vs Mark,713,Toby or Not To Be,Paul,1884,714,Need for Sheed,Mark,1640,2.0,6.0
0,1,Luke vs Jordan,719,He is HIMmelberg,Luke,1872,720,StupidSexyFlanders,Jordan,1682,3.0,5.0
0,1,Lester vs Simon,718,Wilkie Wonka,Simon,1819,717,Baby Got Trac,Lester,1529,4.0,8.0
0,1,Luke vs Jordan,720,StupidSexyFlanders,Jordan,1682,719,He is HIMmelberg,Luke,1872,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,22,Luke vs Simon,719,He is HIMmelberg,Luke,1641,718,Wilkie Wonka,Simon,1702,4.0,3.0
0,23,Jordan vs Simon,720,StupidSexyFlanders,Jordan,1930,718,Wilkie Wonka,Simon,1769,1.0,2.0
0,23,Jordan vs Simon,718,Wilkie Wonka,Simon,1769,720,StupidSexyFlanders,Jordan,1930,2.0,1.0
0,24,Paul vs Jordan,713,Toby or Not To Be,Paul,1911,720,StupidSexyFlanders,Jordan,1840,1.0,2.0


## CURRENTLY LISTED TEAMS

In [32]:
s = httpx.get("https://www.supercoach.com.au/2024/api/afl/draft/v1/leagues/1283/playersStatus", headers=headers, verify=False)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,feed_id,first_name,last_name,team,position,round,points,...,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change,coach_id,coach_team_id,coach_first_name,coach_team_name
0,695,owner,714.0,297373,Marcus,Bontempelli,WBD,1,24,116,...,0,99.82650,1.104762,1.209738,5267.780143,0,21562,714,Mark,Need for Sheed
1,706,owner,718.0,1004592,Tim,English,WBD,31,24,103,...,0,95.86120,0.971698,1.016688,4832.407407,1,8258,718,Simon,Wilkie Wonka
2,458,owner,714.0,996701,Clayton,Oliver,MEL,114,24,0,...,0,86.07190,None,0.88044,4831.194554,-15,21562,714,Mark,Need for Sheed
3,20,owner,718.0,293222,Rory,Laird,ADE,30,24,96,...,0,98.19080,0.864865,1.080776,4961.106309,1,8258,718,Simon,Wilkie Wonka
4,197,owner,716.0,992016,Zach,Merrett,ESS,7,24,135,...,0,99.70260,1.238532,1.104583,5073.702293,-1,71052,716,James,The Gulden Girls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,556,owner,719.0,992330,Jordon,Sweet,PTA,259,24,107,...,0,35.06820,1.080808,0.917343,5202.414353,15,45168,719,Luke,He is HIMmelberg
180,685,owner,720.0,1012210,Matt,Roberts,SYD,147,24,118,...,0,49.16980,1.113208,0.905724,6214.374226,15,180650,720,Jordan,StupidSexyFlanders
181,63,owner,718.0,1014026,Kai,Lohmann,BRL,145,24,93,...,0,21.23920,0.885714,0.778952,5628.682871,5,8258,718,Simon,Wilkie Wonka
182,314,owner,715.0,1025034,Oliver,Dempsey,GEE,117,24,99,...,0,52.09420,0.798387,0.687994,5438.406319,11,10857,715,Anthony,Colonel Sanders


## CURRENT FREE AGENTS + WAIVER

In [33]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,feed_id,first_name,last_name,team,position,round,points,...,ven1,ven2,ven3,livepts,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change
3,459,free_agent,NaN,298210,Christian,Petracca,MEL,238,24,0,...,None,None,None,0,0,62.057000,None,1.084006,5027.559551,-12
39,101,free_agent,NaN,295518,Sam,Docherty,CAR,647,24,0,...,None,None,None,0,0,32.094200,None,0.568182,22488.0,-2
43,719,free_agent,NaN,295467,Jack,Macrae,WBD,244,24,112,...,None,None,None,0,0,68.698900,1.131313,0.838668,4278.777319,15
50,99,free_agent,NaN,996731,Charlie,Curnow,CAR,122,24,0,...,None,None,None,0,0,92.614600,None,0.72336,4802.944609,-16
52,615,free_agent,NaN,294307,Brad,Crouch,STK,626,24,0,...,None,None,None,0,0,31.053300,None,0.717949,9732.142857,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,161,free_agent,NaN,1005107,Nathan,*** Murphy,COL,658,24,0,...,None,None,None,0,0,1.239160,None,None,None,-2
804,684,free_agent,NaN,290188,Sam,*** Reid,SYD,658,24,0,...,None,None,None,0,0,0.743494,None,None,None,-2
805,134,free_agent,NaN,1027021,Josh,*** Carmichael,COL,658,24,0,...,None,None,None,0,0,0.099133,None,None,None,-2
806,723,free_agent,NaN,1024064,Aiden,*** O'Driscoll,WBD,658,24,0,...,None,None,None,0,0,0.495663,None,None,None,-2


In [34]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

In [36]:
s = httpx.get("https://www.supercoach.com.au/2024/api/afl/draft/v1/leagues/1283/processedWaivers", headers=headers, verify=False)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "Waiver"
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,720,2024-03-19T04:00:01+11:00,199,1015873,Archie,Perkins,ESS,181,298421,Jade,Gresham,ESS,Waiver
1,2,716,2024-03-19T04:00:01+11:00,530,280711,Charlie,Dixon,PTA,331,1004938,Gryan,Miers,GEE,Waiver
2,2,717,2024-03-19T04:00:01+11:00,434,296351,Jack,Billings,MEL,457,296420,Alex,Neal-Bullen,MEL,Waiver
3,2,719,2024-03-19T04:00:01+11:00,453,291533,Tom,McDonald,MEL,711,297899,James,Harmes,WBD,Waiver
4,2,714,2024-03-19T04:00:01+11:00,272,993799,Brayden,Fiorini,GCS,592,290627,Dion,Prestia,RIC,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,22,720,2024-08-08T04:00:01+10:00,770,1023492,Harley,Reid,WCE,349,1008139,Toby,Bedford,GWS,Waiver
176,23,718,2024-08-13T04:00:01+10:00,682,290778,Luke,Parker,SYD,754,1023025,Reuben,Ginbey,WCE,Waiver
177,23,720,2024-08-13T04:00:01+10:00,423,998114,Jack,Scrimshaw,HAW,352,1009708,Jack,Buckley,GWS,Waiver
178,24,713,2024-08-20T04:00:01+10:00,202,1001026,Jordan,Ridley,ESS,31,1001195,Izak,Rankine,ADE,Waiver


In [37]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [39]:
s = httpx.get("https://www.supercoach.com.au/2024/api/afl/draft/v1/leagues/1283/trades", headers=headers, verify=False)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "Free Agent"
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev,source
0,1,713,2024-03-12T19:49:52+11:00,526,1006121,Zak,Butters,PTA,NaN,NaN,NaN,NaN,NaN,Free Agent
1,1,713,2024-03-11T19:43:17+11:00,785,1005052,Ethan,Phillips,HAW,461.0,1008541,Kysaiah,Pickett,MEL,Free Agent
2,1,713,2024-03-16T12:45:17+11:00,766,294859,Jeremy,McGovern,WCE,75.0,1002235,Cam,Rayner,BRL,Free Agent
3,1,713,2024-03-16T12:43:54+11:00,322,261510,Tom,Hawkins,GEE,461.0,1008541,Kysaiah,Pickett,MEL,Free Agent
4,1,713,2024-03-12T20:03:00+11:00,681,996765,Tom,Papley,SYD,562.0,1000223,Liam,Baker,RIC,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,22,720,2024-08-06T07:05:39+10:00,480,1015781,Charlie,Comben,NTH,472.0,1017720,Jacob,van Rooyen,MEL,Free Agent
505,22,720,2024-08-08T00:36:44+10:00,204,998215,Will,Setterfield,ESS,184.0,290629,Dyson,Heppell,ESS,Free Agent
506,23,720,2024-08-13T07:07:36+10:00,355,291969,Stephen,Coniglio,GWS,679.0,993905,Callum,Mills,SYD,Free Agent
507,23,720,2024-08-13T07:07:51+10:00,302,994386,Tom,Atkins,GEE,204.0,998215,Will,Setterfield,ESS,Free Agent


In [40]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [43]:
s = httpx.get("https://www.supercoach.com.au/2024/api/afl/draft/v1/leagues/1283/teamtrades", headers=headers, verify=False)
team_trades_json = json.loads(s.text)

In [44]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'Trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

,id,trade_id,player_id,trade_player_id,round,user_team_id,processed,status,source
0,30557,20885,180,68,2,713,2024-03-21T16:54:20+11:00,7,Trade
1,30558,20885,566,615,2,713,2024-03-21T16:54:20+11:00,7,Trade
0,31588,21628,372,54,2,713,2024-03-24T18:55:48+11:00,2,Trade
0,40744,27525,325,257,3,717,2024-04-01T14:58:16+11:00,7,Trade
1,40745,27525,390,0,3,717,2024-04-01T14:58:16+11:00,7,Trade
0,54401,36016,194,719,5,713,2024-04-09T13:55:21+10:00,2,Trade
1,54402,36016,577,598,5,713,2024-04-09T13:55:21+10:00,2,Trade
0,59449,39153,194,663,5,713,2024-04-11T19:26:22+10:00,7,Trade
1,59450,39153,127,154,5,713,2024-04-11T19:26:22+10:00,7,Trade
2,59451,39153,457,369,5,713,2024-04-11T19:26:22+10:00,7,Trade


In [45]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

,round,user_team_id,processed,player_id,source,player.feed_id,player.first_name,player.last_name,player.team.abbrev
0,2,713,2024-03-21T16:54:20+11:00,180,Trade,271129,Todd,Goldstein,ESS
1,2,713,2024-03-21T16:54:20+11:00,566,Trade,993993,Shai,Bolton,RIC
2,2,713,2024-03-21T16:54:20+11:00,615,Trade,294307,Brad,Crouch,STK
3,2,713,2024-03-21T16:54:20+11:00,68,Trade,1005521,Oscar,McInerney,BRL
4,3,717,2024-04-01T14:58:16+11:00,0,Trade,NaN,NaN,NaN,NaN
5,3,717,2024-04-01T14:58:16+11:00,257,Trade,1009199,Noah,Anderson,GCS
6,3,717,2024-04-01T14:58:16+11:00,390,Trade,297354,Karl,Amon,HAW
7,3,717,2024-04-01T14:58:16+11:00,325,Trade,1015889,Max,Holmes,GEE
8,5,713,2024-04-11T19:26:22+10:00,457,Trade,296420,Alex,Neal-Bullen,MEL
9,5,713,2024-04-11T19:26:22+10:00,372,Trade,296347,Josh,Kelly,GWS


## Combined Free Agents, Waiver, and Trades transactions list

In [46]:
columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
149,1,715,2024-02-22T20:30:03+11:00,110,1001398,Matthew,Kennedy,CAR,NaN,NaN,NaN,NaN,NaN,Free Agent
148,1,715,2024-02-22T20:30:03+11:00,116,1023446,Alex,Mirkov,CAR,NaN,NaN,NaN,NaN,NaN,Free Agent
159,1,715,2024-02-22T20:30:06+11:00,110,1001398,Matthew,Kennedy,CAR,NaN,NaN,NaN,NaN,NaN,Free Agent
158,1,715,2024-02-22T20:30:06+11:00,766,294859,Jeremy,McGovern,WCE,NaN,NaN,NaN,NaN,NaN,Free Agent
157,1,715,2024-02-22T20:30:06+11:00,686,1006126,James,Rowbottom,SYD,NaN,NaN,NaN,NaN,NaN,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,24,713,2024-08-21T11:08:52+10:00,124,295461,Adam,Saad,CAR,766.0,294859,Jeremy,McGovern,WCE,Free Agent
95,24,713,2024-08-23T15:22:04+10:00,340,997230,Tyson,Stengle,GEE,99.0,996731,Charlie,Curnow,CAR,Free Agent
96,24,713,2024-08-24T13:14:14+10:00,311,1017063,Toby,Conway,GEE,339.0,280317,Rhys,Stanley,GEE,Free Agent
97,24,713,2024-08-25T05:19:45+10:00,521,1027870,Tom,Anastasopoulos,PTA,704.0,1017126,Sam,Darcy,WBD,Free Agent


In [47]:
df_transactions.to_csv('outputs/transactions.csv')

## Add source to player match results

In [48]:
def get_transaction_status(row):
    post_transaction_list = pd.DataFrame()
    round_number = row['round']
    feed_id = row['player.feed_id']
    source = row['source']
    player_rows = df_player_match_results.loc[(df_player_match_results['round_x'] >= round_number) & (df_player_match_results['feed_id'] == feed_id), :]
    player_rows['source'] = source
    if len(player_rows) > 0:
        df_player_match_results.update(player_rows)

df_player_match_results['source'] = "Draft"
df_transactions.apply(lambda row: get_transaction_status(row), axis=1)
df_player_match_results

,round_x,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points_x,on_field,feed_id,...,ven3,livepts,livegames,owned,points_per_min,total_points_per_min,mvp_value,position_change,round_number_alt,source
0,1.0,71052.0,716.0,James,The Gulden Girls,590.0,RUC,137.0,True,298174,...,None,0.0,0.0,91.251500,1.115789,1.120523,5049.982765,-1.0,R1,Draft
1,2.0,71052.0,716.0,James,The Gulden Girls,590.0,RUC,96.0,True,298174,...,None,0.0,0.0,91.251500,1.115789,1.120523,5049.982765,-1.0,R2,Draft
2,3.0,71052.0,716.0,James,The Gulden Girls,590.0,RUC,73.0,True,298174,...,None,0.0,0.0,91.251500,1.115789,1.120523,5049.982765,-1.0,R3,Draft
3,4.0,71052.0,716.0,James,The Gulden Girls,590.0,RUC,133.0,True,298174,...,None,0.0,0.0,91.251500,1.115789,1.120523,5049.982765,-1.0,R4,Draft
4,5.0,71052.0,716.0,James,The Gulden Girls,590.0,RUC,87.0,True,298174,...,None,0.0,0.0,91.251500,1.115789,1.120523,5049.982765,-1.0,R5,Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,22.0,300791.0,713.0,Paul,Toby or Not To Be,713.0,FWD,0.0,False,1020586,...,None,0.0,0.0,0.495663,0.47619,0.47619,4772.5,22.0,R22,Free Agent
4412,23.0,8258.0,718.0,Simon,Wilkie Wonka,171.0,RUC,85.0,True,1011954,...,None,0.0,0.0,5.452290,0.59434,0.832941,5145.480226,18.0,R23,Free Agent
4413,24.0,8258.0,718.0,Simon,Wilkie Wonka,171.0,RUC,63.0,False,1011954,...,None,0.0,0.0,5.452290,0.59434,0.832941,5145.480226,18.0,R24,Free Agent
4414,24.0,300791.0,713.0,Paul,Toby or Not To Be,216.0,MID,0.0,False,998129,...,None,0.0,0.0,4.560100,None,None,None,-2.0,R24,Free Agent


In [49]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

### Create folder and save all outputs

In [50]:
import os
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
destination = "exports/supercoach_league_scrape_{}_R{}_{}".format(year_number, round_number, timestr)

if not os.path.exists(destination):
    os.makedirs(destination)

df_coaches.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/coach_list.csv".format(year_number, round_number, timestr)) 
df_player_match_results.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/player_match_results.csv".format(year_number, round_number, timestr))
df_ladder.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/ladder.csv".format(year_number, round_number, timestr))
df_fixture_results.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/fixture_results.csv".format(year_number, round_number, timestr))
df_fixture_by_each_team.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/fixture_results_by_team.csv".format(year_number, round_number, timestr))
df_currently_listed_players.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/current_teams.csv".format(year_number, round_number, timestr))
df_available_players.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/current_free_agents_waiver.csv".format(year_number, round_number, timestr))
df_waiver_transactions.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/waiver_transactions.csv".format(year_number, round_number, timestr))
df_fa_transactions.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/free_agent_transactions.csv".format(year_number, round_number, timestr))
df_transactions.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/transactions.csv".format(year_number, round_number, timestr))
df_player_match_results.to_csv("exports/supercoach_league_scrape_{}_R{}_{}/player_match_results.csv".format(year_number, round_number, timestr))